In [ ]:
import json

import pickle

In [ ]:
pkl_file = open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories.pkl', 'rb')

questions = pickle.load(pkl_file)

In [ ]:
path = "/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/questions.jsonl"

questions_list = {}
with open(path, 'r') as f:
    for line in f:
        aa = json.loads(line)
        questions_list.update({aa['question_id']: aa['table_context']})

In [ ]:
q_ids = list(questions.keys())


In [ ]:
table_ids = []
for q_id in q_ids:
    table_ids.append(questions_list[int(q_id.split(':')[0])])

In [ ]:
table_ids

In [ ]:
img_ids_to_path = json.load(open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/image_id_to_refined_path.json', 'r'))
img_ids_to_path_old = json.load(open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/image_id_to_image_path.json', 'r'))

In [ ]:
for k, v in img_ids_to_path_old.items():
    v = v.replace("/home/suyash/final_repo/fetaqa_MM_cleaned/", "")
    v = v.replace("/scratch/jainit_ftq/ftq_images/", "")
    img_ids_to_path_old[k] = v

In [ ]:
import os
all_images = {}
list = []
poster_ids = set()
for k, q in questions.items():
    try:
        all_images.update(
            {q['original_entity'][0][0]: q['original_entity'][0][1]})
        if q['original_entity'][0][1] == 'poster':
            list.append((questions_list[int(k[0])], k[1]))
            poster_ids.add(q['original_entity'][0][0])
        if 'all_images' in q:
            for other in q['all_images']:
                if other[1] == 'poster':
                    poster_ids.add(other[0])
                all_images.update({other[0]: other[1]})
    except:
        all
        # print(q)

In [ ]:
len(poster_ids)

In [ ]:
set(all_images.values())

In [ ]:
query_img_to_path = {}
for k, v in all_images.items():
    if k in poster_ids:
        print(img_ids_to_path[k])
        query_img_to_path.update({k: img_ids_to_path[k]})
    else:
        query_img_to_path.update({k: img_ids_to_path_old[k]})

In [ ]:
query_img_to_path

In [ ]:

prompts_flag_seal = """You are given an image. Do the following: 
1. Analyse the image whether it contains a collage of images. 
if it contains a collage of images, then do the following:
1.1 Identify the number of images in the collage. 
1.2 Identify the type of images in the collage.
1.3 Write the image attributes and objects of each image in json format. Example of json is given under the heading "JSON FOR COLLAGE"
2. if is not a collage of images, then do the following:
2.1 Identify the type of image.
2.2 Write the image attributes and objects in json format. Example of json is given under the heading "JSON FOR SINGLE IMAGE"

JSON FOR COLLAGE
{
  "number_of_images": x,
  "image_1": { "imagetype": "Type of Image whether it is seal, coat of arms, flag, badge, etc", "attribute1":[object1 , object2, object3, ...],"attribute1":[object1 , object2, object3, ...],...,"text" :[ "text in the image"]},
   "image_2": { "imagetype": "Type of Image whether it is seal, coat of arms, flag, badge, etc", "attribute1":[object1 , object2, object3, ...],"attribute1":[object1 , object2, object3, ...],...,"text" :[ "text in the image"]},
   "image_3": { "imagetype": "Type of Image whether it is seal, coat of arms, flag, badge, etc", "attribute1":[object1 , object2, object3, ...],"attribute1":[object1 , object2, object3, ...],...,"text" :[ "text in the image"]}

}

JSON FOR SINGLE IMAGE
{
  "imagetype": "Type of Image whether it is seal, coat of arms, flag, badge, etc",
  "attribute1":[object1 , object2, object3, ...],
  "attribute1":[object1 , object2, object3, ...],
  ...
  "text" : ["text in the image"]
}

Note(Very Important):
You can use the suggested attributes for different image types given below: 
Flag ->  colours, shapes, patterns (stripes, bands or chevrons), mottos, animate objects, inanimate objects
coat of arms -> Colour, shield, Crest(often resembling an animal/object), Motto, animate objects, inanimate objects
seal -> shape, colour, animate objects, inanimate objects
logo -> Shape, colour, animate objects, inanimate objects



Example output for an single image   :
{
  "imagetype": "flag",
  "colours": ["red", "white", "blue"],
  "shapes": ["rectangle"],
  "patterns": ["stripes"],
  "text": ["hail"]
}


Example output for collage  :
{
  "number_of_images": 2,
  "image_1": {
    "imagetype": "flag",
    "colours": ["black", "yellow", "gray"],
    "shapes": ["rectangle"],
    "patterns": ["stripes"],
    "text": ["hail"]
  },
  "image_2": {
    "imagetype": "seal",
    "shape": ["circle"],
    "colour": ["blue"],
    "animate_objects": ["monkey"],
    "inanimate_objects": ["soccer ball"],
    "text": ["Truth Alone Triumphs"]
  }
}
>>> Give the reponse in json format. There is no need to write other info other than json not even ticks.
Also avoid using quotes and new line characters inside json values as it may give json parsing error while doing json.dumps(<your output>)
You should not use instances like {"text":["THis \\n is "quotes in quotes" "]}. 
"""

In [ ]:
prompt_for_poster = """You are given an image of a poster. It can be a movie, song, label any poster. Do the following:
If the image is not a poster, then do the following:
  Return JSON as {"error": "The image is not a poster"}
If the image is a poster, then do the following:
  1. Identify the type of border and frames used in the poster.
  2. Identify the layout of the poster.
  3. Identify the famous celebrities mentioned in the poster.
  4. Identify the fictional characters mentioned in the poster.
  5. Identify the animate objects in the poster.
  6. Identify the inanimate objects in the poster.
  7. Identify the color scheme of the poster.
  8. Identify the text in the image.

JSON FORMAT
{
  "border and frames":["List of borders and frames in the image"],
  "Layout":[(How the visual elements, image and text are arranged in the poster)],
  "Famous celebrities":[List of famous celebrities in the poster],
  "Fictional characters":[List of fictional characters in the poster],
  "animate objects":[List of animate objects in the poster],
  "inanimate objects":[List of inanimate objects in the poster],
  "color scheme":[List of colors used in the poster],
  "text" : [text in the image]
}
>>> Give the reponse in json format. There is no need to write other info other than json not even ticks.
Also avoid using quotes and new line characters inside json values as it may give json parsing error while doing json.dumps(<your output>)
You should not use instances like {"text":["THis \\n is "quotes in quotes" "]}. 
Example output : 
{
  "border and frames": ["Orange border with Black dots"],
  "Layout": ["The poster has a bathtub with a man in it. The bathtub is in a bathroom. The man is wearing a suit and has a drink in his hand. The title of the movie is at the bottom of the poster. The names of the cast are listed below the title."],
  "Famous celebrities": ["Dudley Moore", "Liza Minnelli", "John Gielgud"],
  "Fictional characters": ["Medusa", "Scooby doo"],
  "animate objects": ["Man"],
  "inanimate objects": ["Bathtub", "Drink"],
  "color scheme": ["Brown", "Black", "Orange"],
  "text":[ "Don't you wish you were Arthur?","Arthur"]
}

"""

In [ ]:
prompt_for_landscape = """You are given an image of a landscape. Do the following:
1. Identify the type of landscape.
2. Identify the objects in the landscape.
3. Identify the landmarks in the landscape.
4. Identify the buildings in the landscape.
5. Identify the architecture type in the landscape.   
6. Write the image attributes and objects in json format. Example of json is given under the heading "JSON Template"

JSON Template
{
  "landscape type": "Type of landscape",
  "animate objects":[List of animate objects in the landscape],
  "inanimate objects":[List of inanimate objects in the landscape],
  "recognisable landmarks":[List of recognisable landmarks in the landscape],
  "recognisable buildings":[List of recognisable buildings in the landscape],
  "architecture type":[List of architecture types in the landscape]
}
>>> Give the reponse in json format. There is no need to write other info other than json not even ticks.
Also avoid using quotes and new line characters inside json values as it may give json parsing error while doing json.dumps(<your output>)
You should not use instances like {"text":["THis \\n  is "quotes in quotes" "]}

Example Output :
{
    "landscape type": "City",
    "animate objects": ["People", "Birds"],
    "inanimate objects": ["Buildings", "Cars", "Trees"],
    "recognisable landmarks": ["Erasmusbrug", "De Rotterdam", "Euromast"],
    "recognisable buildings": ["De Rotterdam", "KPN Telecom Building", "Maastoren"],
    "architecture type": ["Modern", "Contemporary"]
}


"""

In [ ]:
poster_images = [k for k, v in all_images.items() if v == 'poster']
poster_paths = {k: v for k, v in query_img_to_path.items()
                if k in poster_images}

In [ ]:
landscape_images = [k for k, v in all_images.items() if v == 'landscape']
landscape_paths = {k: v for k, v in query_img_to_path.items()
                   if k in landscape_images}

In [ ]:
seal_images = [k for k, v in all_images.items() if v !=
               'poster' and v != 'landscape']
seal_paths = {k: v for k, v in query_img_to_path.items() if k in seal_images}

In [ ]:
len(poster_paths), len(landscape_paths), len(seal_paths)

In [ ]:
poster_paths

In [ ]:
# now we will create the prompt file for each image type
prompts = []
PATH = '/home/turning/refactored_images/'
for v in set(poster_paths.values()):

    prompts.append((prompt_for_poster, v))

for v in set(landscape_paths.values()):

    prompts.append((prompt_for_landscape, PATH+v))
    print(PATH + v)

for v in set(seal_paths.values()):

    prompts.append((prompts_flag_seal, PATH+v))
    print(PATH + v)

In [ ]:
# len(prompts), len(poster_paths), len(landscape_paths), len(seal_paths)

In [ ]:
json.dump(prompts, open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories_prompts.json', 'w'))

In [ ]:
# json.dump(prompts[:10], open('/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/ftq_images_test_prompts_10.json', 'w'))

In [ ]:
# #RERUN for landscapes
# landscape_prompts = []
# for  v in set(landscape_paths.values()):
#     landscape_prompts.append((prompt_for_landscape, PATH+v))

In [ ]:
# #json.dump(landscape_prompts, open('/home/turning/Jainit/SM/GptQueries/visual_qs/wtq_landscape_prompts.json', 'w'))

In [ ]:

results = json.load(open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories_prompts.json.results.json', 'r'))

In [ ]:
all_results_dict = {}

for r in results:
    all_results_dict.update({r[0]: r[1]})

In [ ]:
# #json.dump(all_results_dict, open('/home/turning/Jainit/SM/GptQueries/visual_qs/wtq_explict_categories_image_results.json', 'w'))

In [ ]:
def text_to_dict(key, text):
    if text is None:
        return None
    if not isinstance(text, str) and text[1] is None:
        return None
    # take the text from first { to last }
    # Remove the code block syntax if present
    text = text.replace("```json\n", "").replace("\n```", "")
    text = text.replace("```JSON\n", "").replace("\n```", "")
    text = text.replace("```", "")
    # text = text.replace("\n", " ")
    if "This is not a movie poster" in text:
        return {"imagetype": "This is not a movie poster"}
    text = text[text.find("{"):text.rfind("}")+1]
    # Convert the string to a dictionary
    try:
        # Since the input text is expected to be in JSON format, we can use json.loads
        dict_obj = json.loads(text)
        return dict_obj
    except Exception as e:

        # print(key)
        # print(text)

        return text

In [ ]:
parsed_results = {}
None_results = []


None_results = []
for r in all_results_dict.items():
    parsed_dict = text_to_dict(r[0], r[1])
    if parsed_dict == None:
        # print(r[1])
        None_results.append((r[0], r[1]))
    if isinstance(parsed_dict, str):
        print(str(parsed_dict))

    parsed_results.update({r[0]: parsed_dict})

In [ ]:
for k, v in parsed_results.items():
    if not isinstance(v, dict) and v != "" and v is not None:
        print(k)
        print(v)
        print("\n\n\n")

In [ ]:
# parsed_results = json.load(open('/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/od/ftq_explict_categories_image_results_parsed.json', 'r'))

In [ ]:
for k, v in correct_dict.items():
    # if k not in parsed_results:
    parsed_results.update({k: v})

In [ ]:
len(parsed_results)

In [ ]:
# parsed_results = json.load(open('/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/ftq_explict_categories_image_results.json', 'r'))

In [ ]:
# pickle.dump(parsed_results, open('/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories_image_results_parsed.pkl', 'wb'))

In [ ]:
parsed_results = pickle.load(open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories_image_results_parsed.pkl', 'rb'))

In [ ]:
parsed_results.update(json.load(
    open("/home/turning/Jainit/SM/GptQueries/visual_qs/all_results_parsed.json")))

In [ ]:
parsed_results

In [ ]:

prompt_for_uniq1 = """
You are given descriptions of various images. These images are divided into two types - "KEY" image and "NON-KEY" images.
The visual attributes of the "KEY" are listed under the heading "KEY IMAGE ATTRIBUTES" and the visual attributes of the "NON-KEY" images are listed under the heading "NON-KEY IMAGE ATTRIBUTES".
You should do the following:
1. Observe the visual attributes of the "KEY" image and the "NON-KEY" images. Observe all types of attributes especially non-"text" ones.
2. Find all the attributes from all the categories which are present in "KEY" Image but are not present in any of the "NON-KEY" images.
3. Find all such attributes and store them in a json format. Example of json is given under the heading "JSON FORMAT".



>>> Give the reponse in json format. There is no need to write other info other than json not even ticks.
Also avoid using quotes and new line characters inside json values as it may give json parsing error while doing json.dumps(<your output>)
You should not use instances like {"text":["This \\n is "quotes in quotes" "]}.
 
JSON FORMAT: 
{ "Image Type": "<Image Type of Key Image>", "Unique attributes": { "<attribute1>": ["<value1>", "<value2>", "<value3>", ...], "<attribute2>": ["<value1>", "<value2>", "<value3>", ...], ... } }



Example 1: 

KEY Image Attributes: 

{
  "imagetype": "badge",
  "colour": [
    "black",
    "white",
    "yellow"
  ],
  "animate_objects": [
    "monkey"
  ],
  "inanimate_objects": [
    "soccer ball"
  ],
  "text": [
    "Port Vale F.C."
  ]
}
NON-KEY Images Attributes: 

NON-KEY Image 1 Attributes :
 {
  "imagetype": "logo",
  "shape": [
    "circle"
  ],
  "colour": [
    "blue",
    "white",
    "green",
    "yellow",
    "brown"
  ],
  "animate_objects": [
    "tree"
  ],
  "inanimate_objects": [
    "crown",
    "soccer ball"
  ],
  "text": [
    "wigan",
    "athletic",
    "1932"
  ]
}


Unique Attributes : 
{
  "imagetype": "badge",
  
  "Unique attributes": {
  "colour": [
    "black",],
  "animate_objects": [
    "monkey"
  ],
  "inanimate_objects": [],
  "text": [
    "Port Vale F.C."
  ]
  }
}


Example  2: 
KEY Image Attributes 2: 

{
  "landscape type": "City",
  "animate objects": [
    "People",
    "Birds"
  ],
  "inanimate objects": [
    "Buildings",
    "Cars",
    "Trees",
    "Water",
    "Sky"
  ],
  "recognisable landmarks": [
    "Helsinki Cathedral",
    "Suomenlinna",
    "Kiasma Museum of Contemporary Art"
  ],
  "recognisable buildings": [
    "Helsinki Cathedral",
    "Government Palace",
    "Kiasma Museum of Contemporary Art"
  ],
  "architecture type": [
    "Neoclassical",
    "Modern",
    "Contemporary"
  ]
}
NON-KEY Images Attributes : 

NON-KEY Image 1 Attributes :
 {
  "landscape type": "City",
  "animate objects": [
    "People",
    "Birds"
  ],
  "inanimate objects": [
    "Buildings",
    "Cars",
    "Trees",
    "Fountain"
  ],
  "recognisable landmarks": [
    "Aso Rock",
    "National Mosque",
    "Maitama"
  ],
  "recognisable buildings": [
    "National Mosque",
    "CBN Headquarters",
    "NASS Complex"
  ],
  "architecture type": [
    "Modern",
    "Contemporary",
    "Sudanese"
  ]
}
NON-KEY Image 2 Attributes :
 {
  "landscape type": "City",
  "animate objects": [
    "People"
  ],
  "inanimate objects": [
    "Buildings",
    "Monuments"
  ],
  "recognisable landmarks": [
    "Temple of Heaven",
    "Tiananmen Square",
    "Great Wall of China",
    "National Centre for the Performing Arts"
  ],
  "recognisable buildings": [
    "Temple of Heaven",
    "Tiananmen Square",
    "Great Wall of China",
    "National Centre for the Performing Arts"
  ],
  "architecture type": [
    "Ancient Chinese",
    "Modern Chinese"
  ]
}
NON-KEY Image 3 Attributes :
 {
  "landscape type": "City",
  "animate objects": [
    "People"
  ],
  "inanimate objects": [
    "Buildings",
    "Cars",
    "Trees",
    "Statues"
  ],
  "recognisable landmarks": [
    "Charminar",
    "Golconda Fort",
    "Hussain Sagar Lake",
    "Birla Mandir Temple",
    "Qutub Shahi Tombs",
    "Lumbini Park"
  ],
  "recognisable buildings": [
    "Charminar",
    "Golconda Fort",
    "Hussain Sagar Lake",
    "Birla Mandir Temple",
    "Qutub Shahi Tombs",
    "Lumbini Park"
  ],
  "architecture type": [
    "Modern",
    "Contemporary"
  ]
}

Unique Attributes: 

{
  "landscape type": "City",
  "Unique attributes": {
      "recognisable landmarks": [
      "Helsinki Cathedral",
      "Suomenlinna",
      "Kiasma Museum of Contemporary Art"
      ],
      "recognisable buildings": [
      "Helsinki Cathedral",
      "Government Palace",
      "Kiasma Museum of Contemporary Art"
      ],
      "architecture type": [
      "Neoclassical"
      ]
      }
}
Here in the above examples, the unique attributes are such that they are present in "KEY" Image but these attributes are not present in any of the "NON-KEY" images and the attributes are considered. Do not mention any attributes which are not present in the KEY image attributes.
Now based upon above examples perform the task below:

KEY Image Attributes: 

"""

prompt_for_uniq2 = """
NON-KEY Images Attributes: 

"""

Example_string = """
Unique Attributes:

"""

In [ ]:
parsed_dict_from_id = {}
PATH = '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/'
PATH2 = '/home/turning/refactored_images/'
not_in_results = []
for k, v in query_img_to_path.items():
    try:
        if k in poster_ids:
            parsed_dict_from_id.update({k: parsed_results[PATH2+v]})
        else:
            parsed_dict_from_id.update({k: parsed_results[PATH+v]})
    except Exception as e:
        print(e)
        not_in_results.append((k, v))
        print(k, v)

In [ ]:
len(parsed_dict_from_id), len(not_in_results)

In [ ]:
questions[('12054', 'stoke')]

In [ ]:
Unique_attributes_prompts = {}

for question in questions:
    try:
        key_image_id = questions[question]['original_entity'][0][0]
        key_image_attributes = parsed_dict_from_id[key_image_id]
        unique_paths = [query_img_to_path[key_image_id]]

        other_images = []
        for imgs in questions[question]['all_images']:
            if query_img_to_path[imgs[0]] in unique_paths:
                continue
            other_images.append(imgs[0])
            unique_paths.append(query_img_to_path[imgs[0]])
        other_images_attributes = [parsed_dict_from_id[img]
                                   for img in other_images if img in parsed_dict_from_id]
        key_attr_string = json.dumps(key_image_attributes, indent=2)

        other_attr_string = ""
        for i, img in enumerate(other_images):
            other_attr_string += f"NON-KEY Image {i+1} Attributes :\n {json.dumps(parsed_dict_from_id[img], indent=2)}\n"

        prompt = prompt_for_uniq1 + key_attr_string + \
            prompt_for_uniq2 + other_attr_string + Example_string

        Unique_attributes_prompts.update({question[0]+":"+question[1]: prompt})
        # print(prompt)|
        if question[0]+":"+question[1] == '9314:doctor who':
            print(unique_paths)
    except:
        # print(question)
        # print(key_image_id)
        # print(other_images)
        # print(key_image_attributes)
        # print(other_images_attributes)
        # print("\n   \n")
        pass

In [ ]:
questions[('9314', 'doctor who')]

In [ ]:
print(test_prompts['9314:doctor who'])

In [ ]:
json.dump(Unique_attributes_prompts, open("ftq_unique_prompts.json", "w"))

In [ ]:
# create a test dict of unique attribbutes prompts
st = "15673:chicago black hawks"

test_prompts = {}
test_prompts.update({st: Unique_attributes_prompts[st]})
i = 0
for k, v in Unique_attributes_prompts.items():
    test_prompts.update({k: v})
    i += 1
    if i == 100:
        break

In [ ]:
json.dump(test_prompts, open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories_unique_attributes_test_prompts_new.json', 'w'))

In [ ]:
import json
with open('/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories_unique_attributes_test_prompts_new.json', 'r') as f:
    test_prompts = json.load(f)

In [ ]:
print(Unique_attributes_prompts['14619:shaun of the dead'])

In [ ]:
test_prompts_results = json.load(open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories_unique_attributes_test_prompts.json.results.json', 'r'))

In [ ]:
test_prompts_results_dict = {}

for r in test_prompts_results:
    parsed_dict = text_to_dict(r[0], r[1])
    if parsed_dict == None:
        print(r[1])
    if isinstance(parsed_dict, str):
        print(str(parsed_dict))
    print(r[0])
    print(test_prompts[r[0]])
    test_prompts_results_dict.update({r[0]: parsed_dict})

In [ ]:
from pprint import pprint

pprint(test_prompts_results_dict)

In [ ]:
json.dump(Unique_attributes_prompts, open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories_unique_attributes_prompts.json', 'w'))

In [ ]:
for i, (key, prompt) in enumerate(Unique_attributes_prompts.items()):
    print(i, key, prompt)
    print(1)

In [ ]:
unique_attributes = json.load(open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/all_unique_attrs_parsed_latest.json', 'r'))

In [ ]:
unique_attributes_dict = {}

for r in unique_attributes:
    unique_attributes_dict.update({r[0]: text_to_dict(r[0], r[1])})

In [ ]:
len(unique_attributes_dict), len(questions)

In [ ]:
len(unique_attributes_dict)

In [ ]:
for k, v in unique_attributes_dict.items():
    if not isinstance(v, dict) and v != "" and v is not None:
        print(k)
        print(v)
        print("\n\n\n")

In [ ]:
# json.dump(unique_attributes_dict, open(
    # '/home/turning/Jainit/SM/GptQueries/visual_qs/all_unique_attrs_parsed_latest.json', 'w'))

In [ ]:
pprint(unique_attributes_dict)

In [ ]:
question_making_prompt = """
Your task is to rephrase the original question using image attributes of an entity mentioned in the question.
You will be given the original question, the entity mentioned to be replaced, the image attributes of the entity that is to be replaced and also the image type whether it is a landscape, a flag, a poster, a logo, a seal, etc.
Make sure you use these important points: 
1. Use only the mentioned image attributes to rephrase the question. Avoid using any other image attributes which are not given.
3. The rephrased question should be in the form of a question and should be grammatically correct.
3. The rephrased question should be such that the answer does not change. This is very important. 
4. The response should be in JSON format with the rephrased question, the entity and the attributes used to rephrase the question.
5. The rephrased question should be such that it is not too similar to the original question and also the answer the question is trying to ask SHOULD NOT CHANGE.
6. Do not assume any information. Only use the information given in the image attributes. You can return an empty string for the rephrased question if you think it is not possible to rephrase the question using the given image attributes without writing the entity's name.
7. The answer to rephrased question should be same as original question i.e. the semantic meaning of question should not change.
8. You can use any attributes landmarks, buildings, shape, color, animate objects, inanimate objects, fictional characters, etc. mentioned in the image attributes dict to rephrase the question.
The rephrased question should not contain any instance of string "{entity_name}". This is the most crucial thing.
Take a look at the examples below and then proceed to the task.

"""
question_warning = """
>>> Give the reponse in json format. There is no need to write other info other than json not even ticks.
Also avoid using quotes and new line characters inside json values as it may give json parsing error while doing json.dumps(<your output>)
You should not use instances like {\"text\":[\"THis \\n is \"quotes in quotes\" \"]}. 
The json format of your response should be 
"""
question3 = """
{\"Original Question\":Question which is provided to you , \"Rephrased Question\": The rephrased question , \"Entity\": The entity that is to be replaced, \"Used Attributes\":\"The attributes you used to make the rephrased question\" }'

Example 1 Input :

"""

In [ ]:
unique_attributes_dict = json.load(open('/home/turning/Jainit/SM/GptQueries/visual_qs/all_unique_attrs_parsed_latest.json'))

In [ ]:
unique_attributes

In [ ]:
# in unique_attributes_dict, iterate over each value and remove image_type from it

# for k, v in unique_attributes_dict.items():
#     try:
#         for key, value in v.items():
#             if key == "unique_attributes":
#                 if "image_type" in value:
#                     value.pop("image_type")
#                 if "imagetype" in value:
#                     value.pop("imagetype")
#     except:

#         pass

In [ ]:
# json.dump(unique_attributes_dict, open('/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/ftq_uniq_attr.json', 'w'))

In [ ]:
# unique_attributes_dict = json.load(open('/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/ftq_uniq_attr.json', 'r'))

In [ ]:
path = "/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/questions.jsonl"

questions_list = {}
with open(path, 'r') as f:
    for line in f:
        aa = json.loads(line)
        questions_list.update({aa['question_id']: aa['question']})

In [ ]:
unique_attributes_dict["15673:chicago black hawks"]

In [ ]:
questions_list = {str(k): v for k, v in questions_list.items()}

In [ ]:
questions_list['15673']

In [ ]:
question_1 = "20845:liverpool"
# question_1 = "nt-13107:Buick"

questions_list[question_1.split(":")[0]]

In [ ]:
q = questions_list[question_1.split(":")[0]]

In [ ]:
unique_attributes_dict[question_1]

In [ ]:
question2= """"{"Original Question":Which nominations did Fragment receive at the 2014 World Music Awards and what song were they for?, "Entity_TO_BE_REPLACED": "Liverpool", "Image Attributes": {
  "Image Type": "badge",
  "Unique attributes": {
    "colour": [
      "red"
    ],
    "text": [
      "You'll Never Walk Alone",
      "Liverpool",
      "Football Club",
      "Est 1892"
    ],
    "animate_objects": [
      "liver bird"
    ],
    "inanimate_objects": [
      "flames"
    ]
  }
} }

Example 1 Output:

{ "Original Question":Which nominations did Fragment receive at the 2014 World Music Awards and what song were they for?, "Rephrased Question": "When did David Martin play for the football club whose logo has bird and flames, in the Premier League?", "Entity": "Liverpool", "Used Attributes": {"animate objects": ["bird"], "inanimate objects": ["flames"]}}

Example 2 Input:

{"Original Question":How many NHL games did Magnuson play for the Chicago Black Hawks in total, and how many scores did he make?, "Entity_TO_BE_REPLACED": "Chicago Black Hawks", "Image Attributes": {
  "Image Type": "logo",
  "Unique attributes": {
    "colour": [
      "black",
      "green"
    ],
    "animate_objects": [
      "native american"
    ],
    "inanimate_objects": [
      "feathers"
    ]
  }
} }

Example 2 Output:

{ "Original Question":How many NHL games did Magnuson play for the Chicago Black Hawks in total, and how many scores did he make?, "Rephrased Question": "How many NHL games did Magnuson play for the football club whose logo has a head of a native american in total, and how many scores did he make?", "Entity": "Chicago Black Hawks", "Used Attributes": { "animate objects": ["head of a Native American"] }}

Example 3 Input:

{"Original Question":Which nominations did Fragment receive at the 2014 World Music Awards and what song were they for?, "Entity_TO_BE_REPLACED": "world music awards", "Image Attributes": {
  "Image Type": "logo",
  "Unique attributes": {
    "colour": [
      "black"
    ],
    "shape": [
      "rectangle"
    ],
    "text": [
      "world music awards"
    ],
    "animate_objects": [],
    "inanimate_objects": []
  }
}

Example 3 Output:

{ "Original Question":Which nominations did Fragment receive at the 2014 World Music Awards and what song were they for?, "Rephrased Question": "Which nominations did Fragment receive at the event whose logo is rectangular and has black color in it, and what song were they for?", "Entity": "world music awards", "Used Attributes": {"colour": ["black"],"shape":["rectangle"] }}

Example 4 Input: 
{"Original Question": "which is deeper, lake tuz or lake palas tuzla?", "Entity": "Tuzla", "Image Attributes": {
"Image Type": "City",
"Unique attributes": {
"animate objects": [
"Birds"
],
"inanimate objects": [
"Cars",
"Trees"
],
"recognisable landmarks": [
"Spomenik palim borcima NOR-a i \u017ertvama fa\u0161isti\u010dkog terora",
"Banski dvor"
],
"recognisable buildings": [
"Gradska ku\u0107a"
],
"architecture type": [
"Contemporary"
]
}
}}

Example 4 Output:

{ "Original Question": "which is deeper, lake tuz or lake palas tuzla?", "Rephrased Question": "which is deeper, lake tuz or the lake named after the city whose architecture is Contemporary and has Banski dvor as a landmark?", "Entity": "Tuzla", "Used Attributes": {"architecture type": ["Contemporary"], "recognisable landmarks": ["Banski dvor"]}}
As you can see in the above examples in none of the rephrased questions the name of entity to be replaced is mentioned. Now having seen some examples above make a rephrased question using its image attributes given below. 

Input:

"""

In [ ]:
# inputt = f'{{"Original Question":{q}, "Entity_TO_BE_REPLACED": "Liverpool", "Image Attributes": {json.dumps(unique_attributes_dict[question_1], indent=2)} }}'
# outputt = f'{{ "Original Question":{q}, "Rephrased Question": "When did David Martin play for the football club whose logo has bird and flames, in the Premier League?", "Entity": "Liverpool", "Used Attributes": {{"animate objects": ["bird"], "inanimate objects": ["flames"]}}}}'
# question2 = (question2 + inputt+"\n\nExample Output:\n\n" + outputt)
# question2 = (question2 + "\n\n Example Input:\n\n")
# question_2 = "15673:chicago black hawks"
# q = questions_list[question_2.split(":")[0]]
# inputt = f'{{"Original Question":{q}, "Entity_TO_BE_REPLACED": "Chicago Black Hawks", "Image Attributes": {json.dumps(unique_attributes_dict[question_2], indent=2)} }}'
# outputt = f'{{ "Original Question":{q}, "Rephrased Question": "How many NHL games did Magnuson play for the football club whose logo has a head of a native american in total, and how many scores did he make?", "Entity": "Chicago Black Hawks", "Used Attributes": {{ "animate objects": ["head of a Native American"] }}}}'
# question2 = (question2 + inputt+"\n\nExample Output:\n\n" + outputt)
# question2 = (question2 + "\n\n Example Input:\n\n")
# question_3 = "21691:world music awards"
# q = questions_list[question_3.split(":")[0]]
# inputt = f'{{"Original Question":{q}, "Entity_TO_BE_REPLACED": "world music awards", "Image Attributes": {json.dumps({"Image Type": "logo","Unique attributes": {"colour": ["black"],"shape":["rectangle"],"text": ["world music awards"],"animate_objects": [],"inanimate_objects": []}} , indent=2)}'
# outputt = f'{{ "Original Question":{q}, "Rephrased Question": "Which nominations did Fragment receive at the event whose logo is rectangular and has black color in it, and what song were they for?", "Entity": "world music awards", "Used Attributes": {{"colour": ["black"],"shape":["rectangle"] }}}}'
# question2 = (question2 + inputt+"\n\nExample Output:\n\n" + outputt)
# question2 = (question2 + "\n\nAs you can see in the above examples in none of the rephrased questions the name of entity to be replaced is mentioned. Now having seen some examples above make a rephrased question using its image attributes given below. \n\nInput:\n\n")

# print(question2)

In [ ]:
# unique_attributes_dict["17319:mission"]= {"unique_attributes":"Nothing"}

In [ ]:
unique_attributes_dict["13072:ncis: los angeles"]

In [ ]:
all_questions_prompts = {}

for k, v in unique_attributes_dict.items():
    try:
        question_id = k.split(":")[0]
        entity = k.split(":")[1]
        if len(k.split(":")) > 2:
            entity = k.split(":")[1]+":"+k.split(":")[2]
        question = questions_list[question_id]
        image_type = questions[(question_id, entity)]['original_entity'][0][1]
        attributes = unique_attributes_dict[k]
        prompt = question_making_prompt.format(entity_name=entity) + question_warning + question3 + question2 + \
            f'{{"Original Question": "{question}", "Entity": "{entity}", "Image Attributes": {json.dumps(attributes, indent=2)}}}'

        all_questions_prompts.update({k: prompt})
        print(prompt)
    except Exception as error:
        # print(error)
        # print(k, v)
        pass

In [ ]:
json.dump(all_questions_prompts, open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories_question_making_prompts_latest.json', 'w'))

In [ ]:
len(all_questions_prompts)

In [ ]:
import json

In [ ]:
rephrased_questions = json.load(open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories_question_making_prompts.json.results.json', 'r'))

In [ ]:
def text_to_dict(key, text):
    if text is None:
        return None
    if not isinstance(text, str) and text[1] is None:
        return None
    # take the text from first { to last }
    # Remove the code block syntax if present
    text = text.replace("```json\n", "").replace("\n```", "")
    text = text.replace("```JSON\n", "").replace("\n```", "")
    text = text.replace("```", "")
    # text = text.replace("\n", " ")
    if "This is not a movie poster" in text:
        return {"imagetype": "This is not a movie poster"}
    text = text[text.find("{"):text.rfind("}")+1]
    # Convert the string to a dictionary
    try:
        # Since the input text is expected to be in JSON format, we can use json.loads
        dict_obj = json.loads(text)
        return dict_obj
    except Exception as e:

        print(key)
        print(text)

        return text

In [ ]:
rephrased_questions_dict = {}
for r in rephrased_questions:
    parsed_dict = text_to_dict(r[0], r[1])
    question_id = r[0].split(":")[0]
    try:
        rephrased_questions_dict.update(
            {"Original Question": questions_list[str(question_id)]})
    except Exception as e:
        print(question_id)
        print(e)
    rephrased_questions_dict.update({r[0]: parsed_dict})

In [ ]:
for k, v in rephrased_questions_dict.items():
    if not isinstance(v, dict):
        print(k)
        print(v)
        print("\n\n\n")

In [ ]:
for k, v in correct_dict.items():
    rephrased_questions_dict.update({k: v})

In [ ]:
rerun_rephrased_questions_dict = {}
for k, v in rephrased_questions_dict.items():
    if isinstance(v, dict) and 'Rephrased Question' in v and v['Rephrased Question'] == "":
        rerun_rephrased_questions_dict.update({k: all_questions_prompts[k]})
for k, v in rephrased_questions_dict.items():
    if isinstance(v, dict) and 'Rephrased Question' in v and v['Rephrased Question'] == "":
        pprint(v)
        print(k)
        print(unique_attributes_dict[k])
        # break

In [ ]:
json.dump(rephrased_questions_dict, open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/final_questions.json', 'w'))
json.dump(rerun_rephrased_questions_dict, open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories_question_making_prompts_rerun.json', 'w'))

In [ ]:
rerun_rephrased_questions_dict_results = json.load(open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_explict_categories_question_making_prompts_rerun.json.results.json', 'r'))

In [ ]:
for k in rerun_rephrased_questions_dict_results:
    rephrased_questions_dict.update({k[0]: text_to_dict(k[0], k[1])})

In [ ]:
json.dump(rephrased_questions_dict, open(
    '/home/turning/Jainit/SM/GptQueries/visual_qs/ftq/nd/ftq_final_questions.json', 'w'))

In [ ]:
# answers = json.load(open('/home/turning/Jainit/SM/GptQueries/visual_qs/wtq_explict_categories_question_making_prompts_new.json.results.json', 'r'))

In [ ]:
# asnwers_dict = {}
# for a in answers:
#     asnwers_dict.update({a[0]: text_to_dict(a[0], a[1])})

In [ ]:
# json.dump(asnwers_dict, open('/home/turning/Jainit/SM/GptQueries/visual_qs/new_questions_ahh.json', 'w'))